# Classifying Movie Reviews from IMDb

The Internet Movie Database (IMDb) is an online database consisting of movie, TV, and video game information.  Our goal is to analyze a dataset of movie reviews from IMDb and use the Python neural network library [Keras](https://keras.io/) to predict the sentiment of a movie review.  The dataset consists of 25,000 movie reviews from IMDb, labeled by sentiment (positive review or negative review). Reviews have been preprocessed, and each review is encoded as a sequence of word indices (integers).

Words are indexed by overall frequency in the dataset, so for example the integer 1 corresponds to the most frequent word in the dataset, 2 corresponds to the 2nd most frequent word, etc.  Therefore, a sentence is represented by a sequence of integers, and each movie review can be encoded as an integer vector.

In [2]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

## 1. Load and prepare the data

With Keras, this dataset comes [preloaded](https://keras.io/datasets/), so a simple command will allow us to access the training and testing data. The parameter `num_words` specifies the top most frequent words to consider.  The training and testing data are loaded into feature matrices `X_train` and `X_test` (represented as numpy arrays), and the corresponding training/testing labels (`y_train/y_test`) are vectors of binary integers (0 or 1), where 0 represents a bad review and 1 represents a good review.

In [3]:
top_words = 5000

# Loading the data and split it into 50% train, 50% test sets
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

print('number of training examples: ', len(X_train))
print('number of testing examples: ', len(X_test))

number of training examples:  25000
number of testing examples:  25000


We can look at the first training example, which is a movie review encoded as an array of integers

In [4]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


This is how the algorithm will see the review, but it would be nice to see the review in human-readable form as well, so we can get an idea of the review's sentiment.  There is a function in Keras call `get_word_index` that returns a dictionary mapping words to their corresponding indices.  For example, we can have a look at (say, 10) random entries from the dictionary with the restriction that these 10 entries come from the top 5000 most frequent words.

In [32]:
# print out 10 random entries from the top 5000 most frequent words in the word_index dict
word_index = imdb.get_word_index()
rand_indices = np.random.randint(5000, size=10)
for k,v in word_index.items():
    if v in rand_indices:
        print(k,v)

seriously 612
period 807
sue 4717
winter 3438
consistently 4149
the 1
kingdom 4549
brief 1417
contains 1364
particular 840


In [58]:
# see this stackoverflow post:  
# https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset

def get_review(review_number):
    '''
    Put the review back in a form that humans can understand
    '''
    word_to_id = imdb.get_word_index()
    word_to_id = {k:v+3 for k, v in word_to_id.items()}
    word_to_id["<PAD>"] = 0
    word_to_id["<START>"] = 1
    word_to_id["<UNK>"] = 2
    id_to_word = {v:k for k, v in word_to_id.items()}
    print(' '.join( id_to_word[i] for i in X_train[review_number] ))
    
get_review(0)

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they

The neural network model requires that the input vectors all have the same length, so we need to set a fixed length for the inputs and then find a way to deal with the movie reviews that are too long or too short.  This can be done using [sequence.pad_sequences](https://keras.io/preprocessing/sequence/) in the `keras.preprocessing` module, which will truncate longer reviews to a fixed length and also pad shorter reviews with zeros at the end.

In [14]:
max_review_length = 500

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print('train/test set dimensions: ', X_train.shape, X_test.shape, '\n')
print('first sentence in training set, encoded and padded:\n\n ', X_train[0], '\n')
print('first training label: ', y_train[0])

train/test set dimensions:  (25000, 500) (25000, 500) 

first sentence in training set, encoded and padded:

  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

And we'll also one-hot encode the output.

In [ ]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

## 2. Building the  model architecture

We will build a sequential model...

In [3]:
# build the model architecture and compile the model using a loss function (cost function)
# and an optimizer

## 3. Training the model


In [ ]:
# train the model

## 4. Evaluating the model

Let's see the accuracy of the model, along with the precision and recall.

In [ ]:
# score = model.evaluate(x_test, y_test, verbose=0)
# print("Accuracy: ", score[1])